In [2]:
import torch

t = torch.tensor(range(4), dtype=torch.uint8) + 10
z = torch.tensor([[0, 1, 0, 0],
                  [1, 0, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=torch.uint8)

t @ z

tensor([11, 10, 12, 13], dtype=torch.uint8)

In [22]:
import json
import torch
import numpy as np

with open('data/g.json', 'rb') as file:
    data = json.loads(file.read())

program_data = torch.tensor(data['Program'], dtype=torch.uint8)
character_data = torch.tensor(data['Character'], dtype=torch.uint8)

program_data = np.array(data['Program'])

In [4]:
def SEI(state, a, b): state['StatusRegister']['Interrupt'] = 1
def CLD(state, a, b): state['StatusRegister']['Decimal'] = 0

def Z_set(value):
    state['StatusRegister']['Zero'] = 1*(value == 0)

def N_set(value):
    state['StatusRegister']['Negative'] = 1*(value & 0x80 == 0x80)

def LDA(state, a, b): state['Accumulator'] = a; Z_set(a); N_set(a)
def LDX(state, a, b): state['X'] = a;           Z_set(a); N_set(a)
def LDY(state, a, b): state['Y'] = a;           Z_set(a); N_set(a)
def DEX(state, a, b): state['X'] -= 1;          Z_set(state['X']); N_set(state['X'])

def CMP(state, a, b):
    result = state['Accumulator'] - a
    Z_set(result); N_set(result)
    if a <= state['Accumulator']:
        state['StatusRegister']['Carry'] = 1
    else:
        state['StatusRegister']['Carry'] = 0

def STA(state, a, b): state['Memory'][a] = state['Accumulator']
def TXS(state, a, b): state['StackRegister'] = state['X']

def BPL(state, a, b):
    if state['StatusRegister']['Negative'] == 0:
        state['ProgramCounter'] += np.int8(a)

def BCS(state, a, b):
    if state['StatusRegister']['Carry'] == 1:
        state['ProgramCounter'] += np.int8(a)

def BNE(state, a, b):
    if state['StatusRegister']['Zero'] == 0:
        state['ProgramCounter'] += np.int8(a)

def JSR(state, a, b):
    pass

def imm(state, data):
    return data

def abs_read(state, data1, data2):
    print(f'{data2*256 + data1:04x} -> {state["Memory"][data2*0x0100 + data1]}')

    return state['Memory'][data2*0x0100 + data1]

def abs_x_read(state, data1, data2):
    x = state['X']
    print(f'{data2*256 + data1:04x} + {x:02x} -> {state["Memory"][data2*0x0100 + data1 + x]}')

    return state['Memory'][data2*0x0100 + data1 + x]


In [5]:

_ = lambda x: x

instructions = {
    0x00: ["BRK", ["imp"]],
    0x01: ["ORA", ["X", "ind"]],
    0x10: [BPL, imm],
    0x20: [JSR, abs_read, _],
    0x78: [SEI],
    0x8D: [STA, abs_read, _],
    0x9A: [TXS],
    0xA0: [LDY, imm],
    0xA2: [LDX, imm],
    0xA9: [LDA, imm],
    0xAD: [LDA, abs_read, _],
    0xB0: [BCS, imm],
    0xBD: [LDA, abs_x_read, _],
    0xC9: [CMP, imm],
    0xCA: [DEX],
    0xD0: [BNE, imm],
    0xD8: [CLD]
}

In [11]:
state = {
    'ProgramCounter': 0,
    'Accumulator': 0, 'X': 0, 'Y': 0,
    'StackRegister': 0,
    'StatusRegister': {
        'Negative': 0,
        'Zero': 0,
        'Decimal': 0,
        'Interrupt': 0,
        'Carry': 0
    }
}

memory = np.zeros(0x10000, dtype=np.uint8)
memory[0x8000:0x8000+len(program_data)] = program_data
print(len(memory))

state['Memory'] = memory
state['ProgramCounter'] = 0x8000

ops = []

def inc():
    opcode = state['Memory'][state['ProgramCounter']]
    state['ProgramCounter'] += 1

    return opcode

def pc(): return state['ProgramCounter']

def vertical_blank(): state['Memory'][0x2002] = 0x80

vertical_blank()

for _1 in range(47):
    index = pc()
    opcode = inc()

    operation = instructions[opcode.item()]

    if len(operation) == 1:
        op, = operation
        op(state, 0, 0)

        ops.append([index, f'{opcode:02X} {op.__name__}'])

    elif len(operation) == 2:
        op, f1 = operation
        data = inc()
    
        ops.append([index, f'{opcode:02X} {op.__name__} .${data:02x}'])
        op(state, f1(state, data), 0)

    elif len(operation) == 3:
        op, f1, f2 = operation
        data1, data2 = inc(), inc()
        
        if f2 == _:
            ops.append([index, f'{opcode:02X} {op.__name__} ${data2:02x}{data1:02x} ;{data1*0x0100 + data2:04x}'])
            op(state, f1(state, data1, data2), 0)

opcode = state['Memory'][state['ProgramCounter']]

hex(state['ProgramCounter']), state, opcode.item(), hex(opcode.item())

65536
2000 -> 0
2002 -> 128
2002 -> 128
07d7 + 05 -> 0
07d7 + 04 -> 0
07d7 + 03 -> 0
07d7 + 02 -> 0
07d7 + 01 -> 0
07d7 + 00 -> 0
07ff -> 0
90cc -> 162
4011 -> 0


('0x8031',
 {'ProgramCounter': 32817,
  'Accumulator': 0,
  'X': -1,
  'Y': 254,
  'StackRegister': 255,
  'StatusRegister': {'Negative': 0,
   'Zero': 0,
   'Decimal': 0,
   'Interrupt': 1,
   'Carry': 0},
  'Memory': array([  0,   0,   0, ..., 128, 240, 255], dtype=uint8)},
 141,
 '0x8d')

In [12]:
print('\n' + '\n'.join([f'{index:02X} {op}' for index, op in ops]) + '\n')


8000 78 SEI
8001 D8 CLD
8002 A9 LDA .$10
8004 8D STA $2000 ;0020
8007 A2 LDX .$ff
8009 9A TXS
800A AD LDA $2002 ;0220
800D 10 BPL .$fb
800F AD LDA $2002 ;0220
8012 10 BPL .$fb
8014 A0 LDY .$fe
8016 A2 LDX .$05
8018 BD LDA $07d7 ;d707
801B C9 CMP .$0a
801D B0 BCS .$0c
801F CA DEX
8020 10 BPL .$f6
8018 BD LDA $07d7 ;d707
801B C9 CMP .$0a
801D B0 BCS .$0c
801F CA DEX
8020 10 BPL .$f6
8018 BD LDA $07d7 ;d707
801B C9 CMP .$0a
801D B0 BCS .$0c
801F CA DEX
8020 10 BPL .$f6
8018 BD LDA $07d7 ;d707
801B C9 CMP .$0a
801D B0 BCS .$0c
801F CA DEX
8020 10 BPL .$f6
8018 BD LDA $07d7 ;d707
801B C9 CMP .$0a
801D B0 BCS .$0c
801F CA DEX
8020 10 BPL .$f6
8018 BD LDA $07d7 ;d707
801B C9 CMP .$0a
801D B0 BCS .$0c
801F CA DEX
8020 10 BPL .$f6
8022 AD LDA $07ff ;ff07
8025 C9 CMP .$a5
8027 D0 BNE .$02
802B 20 JSR $90cc ;cc90
802E 8D STA $4011 ;1140

